##### 2_compileHospitalMasterCharges

This compiles the Hospital Master Charges published by each hospital that was downloaded in the _1_downloadHospitalMasterCharges.ipynb_ notebook. This notebook also pulls in the health facility data published by **Health Data NY** which is an open data portal for health data published in NY. 

The resultant data prepared from this file is:

1. CMS provider ID to NY Health Facility ID cross walk per hospital
2. Compiled Master Charges Excel file per drg code and drg type. 

The compiled Master Charges file will have:

* Hospital Name
* DRG Type
* fac_id
* providerId
* hospDrg
* description
* avgCharges

In [1]:
import os
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)

1.15.4
0.23.4


In [2]:
# get the facility information
# This dataset was published on NYS Health Open Data
# Dataset Name : Health Facility General Information
# https://health.data.ny.gov/Health/Health-Facility-General-Information/vn5v-hh5r
# This dataset contains the locations of Article 28, Article 36 and Article 40 health care facilities and programs from the Health Facilities Information System (HFIS). 
# Article 28 facilities are hospitals, nursing homes, and diagnostic treatment centers. Article 36 facilities are certified home health care agencies and long term home 
# health care programs. Article 40 facilities are hospices. The dataset currently only contains the locations of hospitals and hospital extension clinics. The data for 
# the remaining facility types will be added in the future.

nysHosp = pd.read_json("https://health.data.ny.gov/resource/7xgt-tyms.json?fac_desc_short=HOSP")

In [3]:
nysHosp.to_csv('dataFiles/nysHealth/nysHospitalsFacilityIds.csv')

In [4]:
# 5 boroughs and counties
nycCounties = ['Queens', 'New York', 'Bronx', 'Kings', 'Richmond']

In [5]:
nycHosp = nysHosp[nysHosp['county'].isin(nycCounties)]

In [6]:
nycHosp.to_csv('dataFiles/nysHealth/nycHospitalsFacilityIds.csv') ## write out a csv copy of the file to archive

The 'nycHospitalsFacilityIds.csv' file was also used to created a cross walk between provider Ids and facility ids. This was done in the googlesheet named "nycHospitalsProviderIds" found [here]( https://docs.google.com/spreadsheets/d/1F8yPe-2uMcAOOzRmYnFenC77GiXnJ0afnRXtBUYe5TQ/edit). This was done mostly using index match or looking at the address between the hospital locations. 

In [7]:
destFol = os.getcwd()+'/dataFiles/rawHospitalChargeData/'
filenames = os.listdir(destFol)

In [8]:
sp = {}
sp['lenoxHillHospital330119'] = 'Lenox Hill CDM Upload'
sp['longIslandJewishMedicalCenter330195'] = 'LIJ - Forest Hills CDM Upload'
sp['statenIslandUniversityHospital330160'] = 'SIUH CDM Upload'
sp['nyuLangoneHospitals330214'] = 'INPATIENT DRG AVERAGE CHARGES'

In [9]:
collect_df = {}

for i in range(len(filenames)):
    if filenames[i].split(".")[-1] == 'csv':
        df_name = filenames[i].split(".")[:-1][0]
        temp = pd.read_csv(destFol+filenames[i])
        collect_df[df_name] = temp
    if (filenames[i].split(".")[-1] in ['xlsx','xls']):
        if(filenames[i].split(".")[:-1][0] in sp):
            df_name = filenames[i].split(".")[:-1][0]
            temp = pd.read_excel(destFol+filenames[i], sheet_name=sp.get(filenames[i].split(".")[:-1][0]))
            collect_df[df_name] = temp
        else:
            df_name = filenames[i].split(".")[:-1][0]
            temp = pd.read_excel(destFol+filenames[i], sheet_name=0)
            collect_df[df_name] = temp

In [10]:
collect_df['jacobiMedicalCenter330127'].rename(columns={'DRG ':'drg',
                                                        'DRG Description':'description',
                                                        'Average Charges':'avgCharges', 
                                                       '# Discharges, Fiscal Year 2017' : 'numDischarges'},
                                               inplace=True)
collect_df['jacobiMedicalCenter330127']['providerId'] = '330127'
collect_df['jacobiMedicalCenter330127'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330127
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330127
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330127
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330127
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330127


In [11]:
collect_df['interfaithMedicalCenter330397'].rename(columns={'DRG':'drg',
                                                        'Description':'description',
                                                        'Average of ChargeTotal2':'avgCharges'},
                                               inplace=True)
collect_df['interfaithMedicalCenter330397']['providerId'] = '330397'
collect_df['interfaithMedicalCenter330397'].head()

,drg,description,avgCharges,providerId
0,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...","$269,744.97",330397
1,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...","$348,709.04",330397
2,5,LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT,"$430,522.31",330397
3,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,"$96,361.39",330397
4,38,EXTRACRANIAL PROCEDURES W CC,"$62,292.35",330397


In [12]:
new_header = collect_df['newYorkPresbyterianQueens330055'].iloc[1]
collect_df['newYorkPresbyterianQueens330055'].drop(collect_df['newYorkPresbyterianQueens330055'].index[[0,1]], inplace = True)
collect_df['newYorkPresbyterianQueens330055'].columns = new_header
collect_df['newYorkPresbyterianQueens330055'].rename(columns={'MS DRG':'drg',
                                                              'Average Charge per Case':'avgCharges'},
                                                     inplace=True)
collect_df['newYorkPresbyterianQueens330055']['providerId'] = '330055'
collect_df['newYorkPresbyterianQueens330055'].head()

1,drg,avgCharges,providerId
2,3,572997,330055
3,4,419103,330055
4,11,212930,330055
5,13,50670.5,330055
6,23,178674,330055


In [13]:
collect_df['mountSinaiBethIsrael330169'].rename(columns={'As of January 2019':'description',
                                                         'Unnamed: 1':'avgCharges'},
                                                inplace=True)
collect_df['mountSinaiBethIsrael330169']['providerId'] = '330169'
collect_df['mountSinaiBethIsrael330169'].head()

,description,avgCharges,providerId
0,.012 MICROFAB NITINOL GUIDWIR,1188.0,330169
1,.012X200 MICROFAB NITINOL GUID,1012.0,330169
2,.014X180 ASAHI PTCA GUIDWRE,150.0,330169
3,.014X180 CONFIANZ PTCA GUIDWRE,200.0,330169
4,.018X180 GLIDEWIRE GUIDEWIRE,350.0,330169


In [14]:
collect_df['coneyIslandHospital330196'].rename(columns={'DRG ':'drg',
                                                        'DRG Description':'description',
                                                        'Average Charges': 'avgCharges',
                                                        '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                     inplace=True)
collect_df['coneyIslandHospital330196']['providerId'] = '330196'
collect_df['coneyIslandHospital330196'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330196
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330196
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330196
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330196
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330196


In [15]:
collect_df['newYorkPresbyterianHospital330101'].drop(collect_df['newYorkPresbyterianHospital330101'].index[[0,1]], inplace = True)

collect_df['newYorkPresbyterianHospital330101'].rename(columns={'NewYork-Presbyterian Average Charge Per Case by MS DRG':'drg',
                                                              'Unnamed: 1':'avgCharges'},
                                                     inplace=True)
collect_df['newYorkPresbyterianHospital330101']['providerId'] = '330101'
collect_df['newYorkPresbyterianHospital330101'].head()

,drg,avgCharges,providerId
2,1,1.67467e+06,330101
3,2,1.08717e+06,330101
4,3,1.40257e+06,330101
5,4,931791,330101
6,5,625730,330101


In [16]:
collect_df['queensHospitalCenter330231'].rename(columns={'DRG NUMBER':'drg',
                                                        'DRG DESCRIPTION':'description',
                                                        'AVERAGE CHARGE':'avgCharges'},
                                               inplace=True)
collect_df['queensHospitalCenter330231']['providerId'] = '330231'
collect_df['queensHospitalCenter330231'].head()

,drg,description,avgCharges,providerId
0,MS001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,"$438,276.00",330231
1,MS003,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...","$1,034,514.00",330231
2,MS004,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...","$549,399.00",330231
3,MS005,LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT,"$683,884.00",330231
4,MS006,LIVER TRANSPLANT W/O MCC,"$398,900.00",330231


In [17]:
collect_df['brookdaleHospitalMedicalCenter330233'].rename(columns={'MS-DRG #':'drg',
                                                                   'MS-DRG':'description',
                                                                   'Discharges':'numDischarges',
                                                                   'Average Charges':'avgCharges'},
                                                          inplace=True)
collect_df['brookdaleHospitalMedicalCenter330233']['providerId'] = '330233'
collect_df['brookdaleHospitalMedicalCenter330233'].head()

,drg,description,numDischarges,avgCharges,providerId
0,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",64,369206.01,330233
1,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",55,255266.63,330233
2,11,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DIAGNOSES ...",2,47901.23,330233
3,12,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DIAGNOSES ...",1,81566.10,330233
4,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,35,136462.68,330233


In [18]:
collect_df['bellevueHospitalCenter330204'].drop(collect_df['bellevueHospitalCenter330204'].index[[0,1]], inplace = True)
collect_df['bellevueHospitalCenter330204'].rename(columns={'The Bellevue Hospital Charge Listing Effective 01/01/2019':'description',
                                                           'Unnamed: 1':'avgCharges'},
                                                          inplace=True)
collect_df['bellevueHospitalCenter330204']['providerId'] = '330204'
collect_df['bellevueHospitalCenter330204'].head()

,description,avgCharges,Unnamed: 2,providerId
2,ISOLATION ROOM,"1,244.00",NaN,330204
3,MS 1ST HR OBS ISOLATION,713.00,NaN,330204
4,MS 1ST HR OBS TELEMETRY,809.00,NaN,330204
5,MS ADD HR OBS ISOLATION,21.00,NaN,330204
6,MS ADD OBS TELEMETRY,24.00,NaN,330204


In [19]:
collect_df['longIslandJewishMedicalCenter330195'].rename(columns={'Charge Description':'description',
                                                                  'Current\nPrice':'avgCharges'},
                                                          inplace=True)
collect_df['longIslandJewishMedicalCenter330195']['providerId'] = '330195'
collect_df['longIslandJewishMedicalCenter330195'].head()

,description,avgCharges,providerId
0,NaN,NaN,330195
1,*CT 3D RECONSTRUC WO WRKSTATON,547.0,330195
2,*CT GUIDANCE NDL PLC SI,1108.0,330195
3,*MR UPR EXT NONJOINT WAW IC LT,3861.0,330195
4,*SC BX ABD RPRTNL MASS,5121.0,330195


In [20]:
collect_df['kingsCountyHospitalCenter330202'].rename(columns={'DRG ':'drg',
                                                        'DRG Description':'description',
                                                        'Average Charges': 'avgCharges',
                                                        '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                     inplace=True)
collect_df['kingsCountyHospitalCenter330202']['providerId'] = '330202'
collect_df['kingsCountyHospitalCenter330202'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330202
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330202
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330202
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330202
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330202


In [21]:
new_header = collect_df['newYorkCommunityHospitalOfBrooklynInc330019'].iloc[2]
collect_df['newYorkCommunityHospitalOfBrooklynInc330019'] = collect_df['newYorkCommunityHospitalOfBrooklynInc330019'][4:3764] #take the data less the header row
collect_df['newYorkCommunityHospitalOfBrooklynInc330019'].columns = new_header #set the header row as the df header
collect_df['newYorkCommunityHospitalOfBrooklynInc330019'].rename(columns={'SITE D DESCRIPTION':'description',
                                                        'SITE RATE': 'avgCharges',
                                                        'COUNT': 'numDischarges'},
                                                     inplace=True)
collect_df['newYorkCommunityHospitalOfBrooklynInc330019']['providerId'] = '330019'
collect_df['newYorkCommunityHospitalOfBrooklynInc330019'].head()

2,SITE,DEPT,SITE CHRG,description,HCPCS CD,avgCharges,REV,OMCR,numDischarges,providerId
4,D,1202,12020200,CRUTCHES,NaN,50.69,429,NaN,NaN,330019
5,D,NaN,22910000,VENIPUNCT/PHLEBTOMY,G0001,34.85,309,NaN,NaN,330019
6,D,NaN,88800001,VENIPUNCT/PHLEBTOMY,36415,34.85,300,NaN,NaN,330019
7,NaN,1202,NaN,NaN,NaN,NaN,NaN,NaN,3,330019
8,D,1204,12040004,A C T H,82024,666.86,301,NaN,NaN,330019


In [22]:
new_header = collect_df['jamaicaHospitalMedicalCenter330014'].iloc[0]
collect_df['jamaicaHospitalMedicalCenter330014'].columns = new_header #set the header row as the df header
collect_df['jamaicaHospitalMedicalCenter330014'] = collect_df['jamaicaHospitalMedicalCenter330014'][2:]
collect_df['jamaicaHospitalMedicalCenter330014'].rename(columns={'DRG Code':'drg',
                                                                 'DRG':'description',
                                                                 'Avg Charge': 'avgCharges'},
                                                        inplace=True)
collect_df['jamaicaHospitalMedicalCenter330014']['providerId'] = '330014'
collect_df['jamaicaHospitalMedicalCenter330014'].head()

,drg,description,nan,avgCharges,providerId
2,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",NaN,320025,330014
3,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",NaN,204911,330014
4,13,"TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES W...",NaN,27746.8,330014
5,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,NaN,87002.1,330014
6,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,43097.9,330014


In [23]:
new_header = collect_df['flushingHospitalMedicalCenter330193'].iloc[0]
collect_df['flushingHospitalMedicalCenter330193'].columns = new_header #set the header row as the df header
collect_df['flushingHospitalMedicalCenter330193'] = collect_df['flushingHospitalMedicalCenter330193'][2:470]
collect_df['flushingHospitalMedicalCenter330193'].rename(columns={'DRG Code':'drg',
                                                                 'DRG ':'description',
                                                                 'Avg Charge': 'avgCharges'},
                                                         inplace=True)
collect_df['flushingHospitalMedicalCenter330193']['providerId'] = '330193'
collect_df['flushingHospitalMedicalCenter330193'].head()

,drg,description,avgCharges,providerId
2,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",370322,330193
3,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",186285,330193
4,38,EXTRACRANIAL PROCEDURES W CC MS,41648.9,330193
5,39,EXTRACRANIAL PROCEDURES W/O CC/MCC MS,36955.4,330193
6,40,PERIPH/CRANIAL NERVE & OTHER NERV SYST PROC W ...,433670,330193


In [24]:
collect_df['lenoxHillHospital330119'].rename(columns={'Charge Description':'description',
                                                      'Current\nPrice': 'avgCharges'},
                                             inplace=True)
collect_df['lenoxHillHospital330119']['providerId'] = '330119'
collect_df['lenoxHillHospital330119'].head()

,description,avgCharges,providerId
0,NaN,NaN,330119
1,*CT GUIDANCE NDL PLC SI,1108.0,330119
2,*CT GUIDANCE TISSUE ABLAT,935.0,330119
3,*ER US OPTH B SCAN RT,360.0,330119
4,*ER US OPTH FRN BODY RT,294.0,330119


In [25]:
collect_df['maimonidesMedicalCenter330194'].rename(columns={'MS-DRG':'drg',
                                                            'Name':'description',
                                                            'Avg Total Charges': 'avgCharges'},
                                                   inplace=True)
collect_df['maimonidesMedicalCenter330194']['providerId'] = '330194'
collect_df['maimonidesMedicalCenter330194'].head()

,drg,description,avgCharges,providerId
0,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1.360593e+06,330194
1,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",6.996321e+05,330194
2,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",4.414728e+05,330194
3,11,"TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES W...",1.612727e+05,330194
4,12,"TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES W CC",1.779713e+05,330194


In [26]:
collect_df['mountSinaiWest330046'].rename(columns={'MS DRG':'drg',
                                                   'MS DRG Description':'description',
                                                   'Average Charges': 'avgCharges'},
                                          inplace=True)
collect_df['mountSinaiWest330046']['providerId'] = '330046'
collect_df['mountSinaiWest330046'].head()

,drg,description,avgCharges,providerId
0,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",959519.03,330046
1,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",1017710.71,330046
2,11,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DX OR LARY...",334399.24,330046
3,12,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DX OR LARY...",265920.02,330046
4,13,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DX OR LARY...",127677.63,330046


In [27]:
new_header = collect_df['hospitalForSpecialSurgery330270'].iloc[5]
collect_df['hospitalForSpecialSurgery330270'].columns = new_header
collect_df['hospitalForSpecialSurgery330270'] = collect_df['hospitalForSpecialSurgery330270'][6:60]
collect_df['hospitalForSpecialSurgery330270'].rename(columns={'DRG#': 'drg',
                                                             'DRG Description': 'description',
                                                             'Average Charges' : 'avgCharges'}, inplace=True)
collect_df['hospitalForSpecialSurgery330270']['providerId'] = '330270'
collect_df['hospitalForSpecialSurgery330270'].head()

5,drg,description,avgCharges,providerId
6,41,PERIPH/CRANIAL NERVE & OTHER NERV SYST PROC W ...,81198.9,330270
7,453,COMBINED ANTERIOR/POSTERIOR SPINAL FUSION W MCC,290629,330270
8,454,COMBINED ANTERIOR/POSTERIOR SPINAL FUSION W CC,182885,330270
9,455,COMBINED ANTERIOR/POSTERIOR SPINAL FUSION W/O ...,130426,330270
10,456,SPINAL FUS EXC CERV W SPINAL CURV/MALIG/INFEC ...,283820,330270


In [28]:
collect_df['nyuLangoneHospitals330214'].rename(columns={'MS-DRG': 'drg',
                                                        'Description': 'description',
                                                        'Average Total Charges' : 'avgCharges'}, inplace=True)
collect_df['nyuLangoneHospitals330214']['providerId'] = '330214'
collect_df['nyuLangoneHospitals330214'].head()

,drg,description,avgCharges,providerId
0,MS001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1698831.13,330214
1,MS002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1013739.91,330214
2,MS003,"ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MO...",1162284.16,330214
3,MS004,"TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NE...",864498.82,330214
4,MS005,LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT,1366837.47,330214


In [29]:
collect_df['statenIslandUniversityHospital330160'].rename(columns={'Current\nPrice': 'avgCharges',
                                                                   'Charge Description': 'description'}, inplace=True)
collect_df['statenIslandUniversityHospital330160']['providerId'] = '330160'
collect_df['statenIslandUniversityHospital330160'].head()

,description,avgCharges,providerId
0,NaN,NaN,330160
1,%7AAD+,25.0,330160
2,*CELL FUNC ASSAY CUINDEX,263.0,330160
3,\ANG REN SSEL 2ORD LT,8048.0,330160
4,10 HYDROXY CARBAZEPINE,102.0,330160


In [30]:
collect_df['woodhullMedicalAndMentalHealthCenter330396'].rename(columns={'DRG ':'drg',
                                                                         'DRG Description':'description',
                                                                         'Average Charges': 'avgCharges',
                                                                         '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                                inplace=True)
collect_df['woodhullMedicalAndMentalHealthCenter330396']['providerId'] = '330396'
collect_df['woodhullMedicalAndMentalHealthCenter330396'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330396
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330396
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330396
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330396
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330396


In [31]:
new_header = collect_df['brooklynHospitalCenterAtDowntownCampus330056'].iloc[4]
collect_df['brooklynHospitalCenterAtDowntownCampus330056'].columns = new_header
collect_df['brooklynHospitalCenterAtDowntownCampus330056'] = collect_df['brooklynHospitalCenterAtDowntownCampus330056'][5:9160]
collect_df['brooklynHospitalCenterAtDowntownCampus330056'].rename(columns={'Charge Description':'description',
                                                                           ' Standard Charges ': 'avgCharges'},
                                                                inplace=True)
collect_df['brooklynHospitalCenterAtDowntownCampus330056']['providerId'] = '330056'
collect_df['brooklynHospitalCenterAtDowntownCampus330056'].head()

4,Charge No,description,avgCharges,Comment,providerId
5,40650025,SPECIMEN HANDLING,31.54,NaN,330056
6,40650034,TISSUE CULTURE-LYMPHOCYTE,97.50,NaN,330056
7,40650038,CHROMOSOME ANAL CT 15-20,602.65,NaN,330056
8,40650045,TISSUE CULTURE SKIN/SOLID,48.81,NaN,330056
9,03610013,CYSTO LOCAL 0.5 HOUR,"6,640.00",NaN,330056


In [32]:
collect_df['nYEyeAndEarInfirmary330100'].rename(columns={'MS DRG': 'drg',
                                                         'MS DRG Description': 'description',
                                                         'Average Charge' : 'avgCharges'}, inplace=True)
collect_df['nYEyeAndEarInfirmary330100']['providerId'] = '330100'
collect_df['nYEyeAndEarInfirmary330100'].head()

,drg,description,avgCharges,providerId
0,12,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DX OR LARY...",216217,330100
1,13,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DX OR LARY...",122914,330100
2,60,MULTIPLE SCLEROSIS & CEREBELLAR ATAXIA W/O CC/MCC,27123,330100
3,114,ORBITAL PROCEDURES W/O CC/MCC,16425,330100
4,115,EXTRAOCULAR PROCEDURES EXCEPT ORBIT,45603,330100


In [33]:
collect_df['elmhurstHospitalCenter330128'].rename(columns={'DRG ':'drg',
                                                           'DRG Description':'description',
                                                           'Average Charges': 'avgCharges',
                                                           '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                  inplace=True)
collect_df['elmhurstHospitalCenter330128']['providerId'] = '330128'
collect_df['elmhurstHospitalCenter330128'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330128
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330128
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330128
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330128
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330128


In [34]:
collect_df['stBarnabasHospital330399'].rename(columns={'Description/FIM Description':'description',
                                                       'Primary Price': 'avgCharges'},
                                              inplace=True)
collect_df['stBarnabasHospital330399']['providerId'] = '330399'
collect_df['stBarnabasHospital330399'].head()

,description,avgCharges,providerId
0,1 ICS FACE MASK,156.0,330399
1,1 ICS GIRDLE,169.0,330399
2,1 ICS GLOVES,127.0,330399
3,1 ICS KNEE LENGTH,88.0,330399
4,1 ICS VEST,171.0,330399


In [35]:
collect_df['lincolnMedicalMentalHealthCenter330080'].rename(columns={'DRG ':'drg',
                                                           'DRG Description':'description',
                                                           'Average Charges': 'avgCharges',
                                                           '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                  inplace=True)
collect_df['lincolnMedicalMentalHealthCenter330080']['providerId'] = '330080'
collect_df['lincolnMedicalMentalHealthCenter330080'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330080
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330080
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330080
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330080
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330080


In [36]:
collect_df['richmondUniversityMedicalCenter330028'].rename(columns={'DRG':'drg', 
                                                                   'Description':'description',
                                                                   'Average Charge': 'avgCharges'}, 
                                                           inplace=True)
collect_df['richmondUniversityMedicalCenter330028']['providerId'] = '330028'
collect_df['richmondUniversityMedicalCenter330028'].head()

,drg,description,avgCharges,providerId
0,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...","$501,793.93",330028
1,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...","$290,539.65",330028
2,12,"TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES W CC","$92,176.26",330028
3,23,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,"$229,481.24",330028
4,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,"$140,201.16",330028


In [37]:
collect_df['metropolitanHospitalCenter330199'].rename(columns={'DRG ':'drg',
                                                               'DRG Description':'description',
                                                               'Average Charges': 'avgCharges',
                                                               '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                      inplace=True)
collect_df['metropolitanHospitalCenter330199']['providerId'] = '330199'
collect_df['metropolitanHospitalCenter330199'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330199
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330199
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330199
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330199
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330199


In [38]:
collect_df['universityHospitalOfBrooklynDownstate330350'].rename(columns={'MS- DRG': 'drg',
                                                                          'DRG DESCRIPTION': 'description',
                                                                          'AVG. CHRGS' : 'avgCharges'},
                                                                 inplace=True)
collect_df['universityHospitalOfBrooklynDownstate330350']['providerId'] = '330350'
collect_df['universityHospitalOfBrooklynDownstate330350'].head()

,drg,description,avgCharges,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,providerId
0,1.0,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,371490.720000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330350
1,3.0,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",188458.413333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330350
2,4.0,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",183722.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330350
3,11.0,"TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES W...",68974.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330350
4,26.0,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,37893.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330350


In [39]:
new_header = collect_df['wyckoffHeightsMedicalCenter330221'].iloc[1]
collect_df['wyckoffHeightsMedicalCenter330221'].columns = new_header
collect_df['wyckoffHeightsMedicalCenter330221'] = collect_df['wyckoffHeightsMedicalCenter330221'][2:]
collect_df['wyckoffHeightsMedicalCenter330221'].rename(columns={'ITEM/PROCEDURE DESCRIPTION': 'description',
                                                                'PRICE' : 'avgCharges'},
                                                       inplace=True)
collect_df['wyckoffHeightsMedicalCenter330221']['providerId'] = '330221'
collect_df['wyckoffHeightsMedicalCenter330221'].head()

1,description,avgCharges,providerId
2,PRV'TE RM & BRD - 6TH FL,3500,330221
3,PRV'TE RM & BRD - 9TH FL,3500,330221
4,PRV'TE RM & BRD - ED HD,3500,330221
5,PRV'TE RM & BRD - 5TH FL,3500,330221
6,PRV'TE RM & BRD - PEDS,3500,330221


In [40]:
new_header = collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'].iloc[1]
collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'].columns = new_header
collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'] = collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'][2:]
collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'].rename(columns={'MS DRG': 'drg',
                                                                                 'Average Charge per Case' : 'avgCharges'},
                                                                        inplace=True)
collect_df['newYorkPresbyterianBrooklynMethodistHospital330236']['providerId'] = '330236'
collect_df['newYorkPresbyterianBrooklynMethodistHospital330236'].head()

1,drg,avgCharges,providerId
2,3,580589,330236
3,4,333203,330236
4,11,176173,330236
5,12,125676,330236
6,20,543590,330236


In [41]:
collect_df['harlemHospitalCenter330240'].rename(columns={'DRG ':'drg',
                                                         'DRG Description':'description',
                                                         'Average Charges': 'avgCharges',
                                                         '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                inplace=True)
collect_df['harlemHospitalCenter330240']['providerId'] = '330240'
collect_df['harlemHospitalCenter330240'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330240
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330240
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330240
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330240
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330240


In [42]:
collect_df['northCentralBronxHospital330385'].rename(columns={'DRG ':'drg',
                                                              'DRG Description':'description',
                                                              'Average Charges': 'avgCharges',
                                                              '# Discharges, Fiscal Year 2017': 'numDischarges'},
                                                     inplace=True)
collect_df['northCentralBronxHospital330385']['providerId'] = '330385'
collect_df['northCentralBronxHospital330385'].head()

,drg,description,avgCharges,numDischarges,providerId
0,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCED...,430171.29,285,330385
1,5,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,348734.66,260,330385
2,20,CRANIOTOMY FOR TRAUMA,105928.76,127,330385
3,21,CRANIOTOMY EXCEPT FOR TRAUMA,101825.67,271,330385
4,22,VENTRICULAR SHUNT PROCEDURES,57342.91,25,330385


In [43]:
collect_df['montefioreMedicalCenter330059'].rename(columns={'MS DRG':'drg',
                                                           'DRG Title':'description',
                                                           'Avg Charges': 'avgCharges',
                                                           'Case': 'numDischarges'},
                                                  inplace=True)
collect_df['montefioreMedicalCenter330059']['providerId'] = '330059'
collect_df['montefioreMedicalCenter330059'].head()

,drg,description,Weight,Cases,avgCharges,providerId
0,1,HEART TRANSPLANT/IMPLNT HEART ASST SYS W MCC,25.5878,69,1259802,330059
1,2,HEART TRANSPLNT/IMPLNT HEART ASST SYS W/O MCC,15.2462,1,594968,330059
2,3,ECMO/TRCH W MV 96+H/PDX EX FCE MTH&NCK &MJ OR,17.7135,242,1073311,330059
3,4,TRCH W MV 96+H/ PDX EXC FCE MTH&NCK NO MJ OR,11.1213,154,828178,330059
4,5,LIVER TRANSPLANT W MCC OR INTESTINL TRANSPLNT,10.4106,26,821231,330059


In [44]:
collect_df['mountSinaiHospital330024'].rename(columns={'MS DRG':'drg',
                                                       'MS DRG Description':'description',
                                                       'Average Charges': 'avgCharges'},
                                              inplace=True)
collect_df['mountSinaiHospital330024']['providerId'] = '330024'
collect_df['mountSinaiHospital330024'].head()

,drg,description,avgCharges,providerId
0,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1132881,330024
1,2,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,589282,330024
2,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",1144216,330024
3,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",615856,330024
4,5,LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT,651207,330024


In [45]:
new_header = collect_df['bronxLebanonHospitalCenter330009'].iloc[1]
collect_df['bronxLebanonHospitalCenter330009'].columns = new_header
collect_df['bronxLebanonHospitalCenter330009'].drop(collect_df['bronxLebanonHospitalCenter330009'].index[[0,1]], inplace = True)
collect_df['bronxLebanonHospitalCenter330009'].rename(columns={'DRG': 'drg', 
                                                              'DRG DESCRIPTION' : 'description',
                                                              'AVERAGE CHARGE': 'avgCharges'}, 
                                                      inplace = True)
collect_df['bronxLebanonHospitalCenter330009']['providerId'] = '330009'
collect_df['bronxLebanonHospitalCenter330009'].head()

1,drg,description,avgCharges,providerId
2,775,ALCOHOL ABUSE & DEPENDENCE,22066.8,330009
3,772,ALCOHOL/DRUG DEP W/REHAB DETOX,29814.1,330009
4,198,ANGINA PECTORIS & CORON ATHERO,21846.4,330009
5,141,ASTHMA,9181.95,330009
6,753,BIPOLAR DISORDERS,44274.2,330009


In [204]:
allNycHospitals = pd.concat(collect_df, ignore_index=True, keys=None, sort=False)

In [205]:
# drop the columns not needed
allNycHospitals.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3', 'Unnamed: 4',
                      'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True)

In [206]:
allNycHospitals.drop(allNycHospitals.columns[10], axis=1, inplace=True)

In [207]:
allNycHospitals.columns

Index([  'description',    'avgCharges',    'providerId',           'drg',
       'numDischarges',     'Charge No',       'Comment',             nan,
             ' Weight',        ' Cases',          'DEPT',     'SITE CHRG',
            'HCPCS CD',           'REV',          'OMCR'],
      dtype='object')

In [208]:
allNycHospitals['temp'] = allNycHospitals['drg']

In [209]:
allNycHospitals.loc[allNycHospitals['drg'].str.startswith('MS') == True, 'temp'] = allNycHospitals['drg'].str[2:]

In [210]:
# drop the row with erroneous grand total
allNycHospitals = allNycHospitals[(allNycHospitals['drg'] != 'Grand Total') &
                                  (allNycHospitals['avgCharges'] != 'Contact hospital for price as this is based on the number of units needed. ') &
                                   (allNycHospitals['avgCharges'].isnull() == False) &
                                 (allNycHospitals['avgCharges'] != ' Keep? NO ') &
                                 (allNycHospitals['avgCharges'] != 'Price')]

In [211]:
allNycHospitals['hospDrg'] = pd.to_numeric(allNycHospitals['temp'])

In [212]:
allNycHospitals['temp2'] = allNycHospitals['avgCharges'].astype(str)

In [213]:
allNycHospitals.loc[allNycHospitals['avgCharges'].str.startswith('$') == True, 'temp2'] = allNycHospitals['avgCharges'].replace('[\$,)]','', regex=True)

In [214]:
allNycHospitals['temp2'] = allNycHospitals['temp2'].replace('[\$,)]','', regex=True)

In [215]:
allNycHospitals['avgCharges'] = pd.to_numeric(allNycHospitals['temp2'])

In [219]:
allNycHospitals[['providerId', 'hospDrg', 'description', 'avgCharges']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96512 entries, 0 to 96799
Data columns (total 4 columns):
providerId     96512 non-null object
hospDrg        12549 non-null float64
description    94372 non-null object
avgCharges     96512 non-null float64
dtypes: float64(2), object(2)
memory usage: 6.2+ MB


In [222]:
xWalk = pd.read_csv(os.getcwd()+'/dataFiles/nycHospitalsProviderIdFacilityIdCrossWalk.csv')

In [227]:
xWalk['providerId'] = xWalk['Provider ID'].astype(str)

In [235]:
compiledMasterchargesNycHospitals = xWalk.merge(allNycHospitals[['providerId', 'hospDrg', 'description', 'avgCharges']], on = 'providerId')

In [237]:
compiledMasterchargesNycHospitals.drop(['Unnamed: 0', 'Provider ID', 'Provider ID', 'C-Section?'], 
                                      axis=1, inplace=True)

In [238]:
compiledMasterchargesNycHospitals.to_csv(os.getcwd()+'/dataFiles/compiledMasterchargesNycHospitals.csv')

In [239]:
compiledMasterchargesNycHospitals.head()

,Hospital Name,DRG Type,fac_id,providerId,hospDrg,description,avgCharges
0,BRONX-LEBANON HOSPITAL CENTER,APR-DRG,1164,330009,775.0,ALCOHOL ABUSE & DEPENDENCE,22066.805466
1,BRONX-LEBANON HOSPITAL CENTER,APR-DRG,1164,330009,772.0,ALCOHOL/DRUG DEP W/REHAB DETOX,29814.074395
2,BRONX-LEBANON HOSPITAL CENTER,APR-DRG,1164,330009,198.0,ANGINA PECTORIS & CORON ATHERO,21846.361053
3,BRONX-LEBANON HOSPITAL CENTER,APR-DRG,1164,330009,141.0,ASTHMA,9181.951122
4,BRONX-LEBANON HOSPITAL CENTER,APR-DRG,1164,330009,753.0,BIPOLAR DISORDERS,44274.231067
